# Exploring StackOverflow!

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

## Populating the Graphs

Each graph is build independantly from the provided `.txt` files of temporal network of interactions. \
Users are represented as nodes and answers\comments as edges.

The design choices of the following method are:
- Using directed graphs. 
- *Simple* graphs, there are no loops in the graphs. Users who answer to themselves are discarded cases.
- Only one attributes is assigned to the edges: weight. The weights are:
    - `1` for answers to questions
    - `2/3` for comments on questions
    - `1/2` for comments to answers
- Time resolution is one day.
- The graphs are build given a time input to avoid such attribute for the sake of simplicity and robustess.
---

In [3]:
def from_time_int_to_dates(time_interval):
    # Convert the time interval in start and end dates 
    time_interval = tuple(map(datetime.fromisoformat, time_interval)) # converting time interval into datetime format
    time_interval = tuple(map(datetime.timestamp, time_interval)) # converting time interval into POSIX timestamp 
    start_d = int(time_interval[0]) #converting to string to compare with the txt
    end_d = int(time_interval[1])
    return start_d, end_d

In [268]:
def get_full_graph(file):
    
    # Initialize the directed graph
    G = nx.DiGraph()
    
    # Create mapping of files
    map_files = {1: "sx-stackoverflow-a2q.txt", 2:"sx-stackoverflow-c2q.txt", 3:"sx-stackoverflow-c2a.txt"}
    
    # Select the file chosen, open it and read the lines
    with open(map_files[file], "r", encoding="UTF-8") as f:
        for line in tqdm(f.readlines()):
            
            # Parse the line
            elems = line.split(' ')
            
            # Add the edge to the graph if it is not present
            if (elems[0], elems[1]) not in G.edges():
                G.add_edge(elems[0], elems[1])
                    
    return G

In [ ]:
graph1_full = get_full_graph(1)

In [ ]:
graph2_full = get_full_graph(2)

In [ ]:
graph3_full = get_full_graph(3)

In [ ]:
print(graph1_full)
print(graph2_full)
print(graph3_full)

#### Write the 3 full graphs into files to save computation each time

In [ ]:
nx.write_gml(graph1_full, "graph1_full")

In [ ]:
nx.write_gml(graph2_full, "graph2_full")

In [ ]:
nx.write_gml(graph3_full, "graph3_full")

#### Read the 3 full graphs from files

In [ ]:
nx.read_gml("graph1_full")
nx.read_gml("graph2_full")
nx.read_gml("graph3_full")

In [4]:
def get_graph(time_interval, file = 3):
    
    # Initialize the graph
    G = nx.DiGraph()
    
    # Create mapping of files and mapping of weights
    map_files = {1: "sx-stackoverflow-a2q.txt", 2:"sx-stackoverflow-c2q.txt", 3:"sx-stackoverflow-c2a.txt"}
    map_weights = {1: 1.0, 2: 2/3, 3: 1/2}
    
    # Get the start and end dates 
    start, end = from_time_int_to_dates(time_interval)
    
    # Select the file chosen, open it and read the lines
    with open(map_files[file], "r", encoding="UTF-8") as f:
        for line in tqdm(f.readlines()):
            
            # Parse the line
            elems = line.split(' ')
            
            # Add to the graph if it is in the time interval
            if start <= int(elems[2]) <= end:
                # If the edge already exists --> increment the weight, else simply add the new edge
                if (elems[0], elems[1]) in G.edges():
                    G[elems[0]][elems[1]]['weight'] += float(map_weights[file])
                else:
                    G.add_edge(elems[0], elems[1], weight = float(map_weights[file]))
                    
    return G

In [5]:
interval = ("2008-11-01","2008-11-10")

In [6]:
graph1 = get_graph(time_interval=interval, file=1)

100%|██████████| 17823525/17823525 [00:40<00:00, 441142.66it/s]


In [7]:
graph2 = get_graph(time_interval=interval, file=2)

100%|██████████| 20268151/20268151 [00:51<00:00, 394643.74it/s]


In [8]:
graph3 = get_graph(time_interval=interval, file=3)

100%|██████████| 25405374/25405374 [01:09<00:00, 363701.02it/s]


In [11]:
print(graph1)
print(graph2)
print(graph3)

DiGraph with 4668 nodes and 10758 edges
DiGraph with 1111 nodes and 1177 edges
DiGraph with 3070 nodes and 5990 edges


In [9]:
d1 = nx.to_dict_of_dicts(graph1)
d2 = nx.to_dict_of_dicts(graph2)
d3 = nx.to_dict_of_dicts(graph3)

### Merging the graphs

This function will merge two graphs that were obtained by the function **get_graph()** with the same time interval. 

In [10]:
def merged_graph(graph_1, graph_2):
    
    # Iterate over the edges from the second graph
    for edge_2 in graph_2.edges(data = True):
        # If the edge of graph 2 is also in graph 1, only sum weights
        if (edge_2[0],edge_2[1]) in graph_1.edges():
            graph_1[edge_2[0]][edge_2[1]]['weight'] += float(edge_2[2]['weight'])
        # Else add the edge of graph 2 also in graph 1
        else:
            graph_1.add_edge(edge_2[0], edge_2[1], weight = float(edge_2[2]['weight']))
            
    return graph_1

In [12]:
merged = merged_graph(graph1, graph2)
merged = merged_graph(merged, graph3)

In [13]:
print(merged)

DiGraph with 5134 nodes and 17414 edges


In [14]:
d_merge = nx.to_dict_of_dicts(merged)

### Functionality 1 - Get the overall features of the graph

Graph density/sprcity is computed as defined in *"Introduction to Algorithms" by Cormern, Leiserson, Rivest, and Stein*: \
A graph $G = (E, V)$, with $E$ denoting the edges and $V$ denoting the vertices,  is sparse if $|E| << |V|^2$ and dense if $|E|$ is close to $|V|$. \
And so if $|E|$ differs an order of magnitude from $|V|$ the graph is considered sparse, otherwise it is dense.
The density degree expression is:

\begin{equation}
\frac{|V|}{|E|(|E| - 1)} \approx \frac{|V|}{|E|^2}
\end{equation}

In [15]:
# Input: a graph as a dict of dicts
# Output: a boolean True/False

def directed(dict_graph):
    direct = False
    # Iterating through the dictionary items
    for node, neighbours in dict_graph.items():
        # For each list of adjacency of node
        for neighbour in list(neighbours.keys()):
            # If the node is not present in the adj list of his neighbours --> break
            if node not in list(dict_graph[neighbour].keys()):
                direct = True
                break
    return direct

In [16]:
# Input: a graph as a dict of dicts
# Output: number of nodes in the graph

def n_users(dict_graph):
    # Return number of keys
    return len(dict_graph)

In [17]:
# Input: a graph as a dict of dicts
# Output: number of edges in the graph

# For this implementation we considered the "out-degree" = number of edges coming out from the node 
def n_interactions(dict_graph):
    # Sum the length of lists of edges
    n_int = 0
    for neighbour in dict_graph.values():
        n_int += len(list(neighbour.keys()))
    return n_int

In [18]:
# Input: a graph as a dict of dicts
# Output: average number of links for nodes

def average_n_links(dict_graph):
    return n_interactions(dict_graph) / n_users(dict_graph)

In [19]:
def F1_OverallFeatures(dict_graph):
    """
    Input: One of the 3 graphs in dictionary format

    Output:
      Whether the graph is directed or not
      Number of users
      Number of answers/comments
      Average number of links per user
      Density degree of the graph
      Whether the graph is sparse or dense
    """
    
    # quering if the graph is directed
    Directed = directed(dict_graph)
    
    # using a variable to print the output
    if Directed: 
        IsDirectedPrint = 'directed'
    else:
        IsDirectedPrint = 'undirected'
        
    # computing the number of users
    NofUsers = n_users(dict_graph) 
    
    # computing the number of interactions
    NofInteractions = n_interactions(dict_graph)
    
    # computing the avarege number of links per user
    AvgUserLinks = average_n_links(dict_graph)
    
    # computing density degree
    DensityDegree = NofInteractions / NofUsers**2
    
    # evaluating sparsity/density
    if NofUsers / NofInteractions**2 < 10:
        SparseDense = 'sparse'
    else:
        SparseDense = 'dense'
        
    # print the results
    print(f"The input graph is -> {IsDirectedPrint}")
    print(f"Number of users -> {NofUsers}")
    print(f"Number of answers/comments -> {NofInteractions}")
    print(f"Average number of links per user -> {AvgUserLinks:.2}")
    print(f"Density degree -> {DensityDegree: .2}")
    print(f"The graph is -> {SparseDense}")

In [20]:
F1_OverallFeatures(d1)

The input graph is -> directed
Number of users -> 4668
Number of answers/comments -> 10758
Average number of links per user -> 2.3
Density degree ->  0.00049
The graph is -> sparse


### Functionality 2 - Find the best users!

#### Degree centrality
As defined in *Introduction to Graph Concepts for Data Science of Aris Anagnostopoulos*, the normalized degree centrality of node $v$ is:

\begin{equation}
\frac{d_v}{|V|-1} \approx \frac{d_v}{|V|}
\end{equation}

with $d_v$ the degree of the node $v$, i.e. the number of edges incident to the node.

In [21]:
# Calculate the degree as "out-degree"
def degree(dict_graph, node):
    return len(list(dict_graph[node].values()))

In [22]:
# Calculate the degree as "in-degree"
def in_degree(dict_graph, node):
    out = 0
    for u in list(dict_graph.keys()):
        if u != node and (node in list(dict_graph[u].keys())):
            out += 1
    return out

In [23]:
# Apply the definition
def degree_centrality(dict_graph, node):
    out = degree(dict_graph, node) / n_interactions(dict_graph)
    print(f"{out:.2}")

In [24]:
degree_centrality(d1, "30155")

0.0018


In [25]:
degree_centrality(d1, "4120")

9.3e-05


#### Closeness centrality
As defined in *Introduction to Graph Concepts for Data Science of Aris Anagnostopoulos*, the normalized closeness centrality of node $v$ is:


\begin{equation}
\frac{|V|-1}{\sum_{u \epsilon V} d(v,u)} \approx \frac{|V|}{\sum_{u \epsilon V} d(v,u)}
\end{equation}

with $d(v,u)$ the distance between nodes $v$ and $u$, that is the length of a shortest path between $v$ and $u$.\
As a design choice $d(v,u)$ is taken as the inverse of the weight of the edge of $(v,u)$, this leads to an inverse relationship between interaction and distances: the more a user posts, the closer he gets to the comunity.  

In [26]:
def get_outgoing_edges(dict_graph, node):
    return list(dict_graph[node].keys())

In [27]:
# Input: a graph as a dict of dicts, the start node
# Output: two dictionaries: the first memorize the nodes involved in the shortest paths, the second is the shortest path in distance terms.
# Pseudocode from: https://en.wikipedia.org/wiki/Dijkstra's_algorithm#Pseudocode

def shortest_path_Dijkstra(dict_graph, node):
    
    # Initialize two dictionaries to track visited nodes and weights
    nodes_dist = dict.fromkeys(dict_graph.keys(), float('inf'))
    
    # Initialize another dictionary that track the nodes in the shortest path
    previous_nodes = dict.fromkeys(dict_graph.keys())
    
    # Create a list of the unvisited nodes 
    unvisited = list(dict_graph.keys())
    
    # First update on the selected node
    nodes_dist[node] = 0
    
    # Start iterating until each node is visited
    while(unvisited):
        
        # Select the current min_node with a for-loop
        min_node = None
        for n in unvisited:
            if min_node == None:
                min_node = n
            elif nodes_dist[n] < nodes_dist[min_node]:
                min_node = n
            
        # Update distances of the current min_node's neighbors
        neighbors = get_outgoing_edges(dict_graph, min_node)
        for neighbor in neighbors:
            # Calculate the possible update to the shortest path and check if it is < then the actual distance
            possible_update = nodes_dist[min_node] + dict_graph[min_node][neighbor]['weight']
            if possible_update < nodes_dist[neighbor]:
                nodes_dist[neighbor] = possible_update
                # We also update the best path to the current node
                previous_nodes[neighbor] = min_node
        
        # Remove the current min_node from the unvisited list
        unvisited.remove(min_node)
    
    return previous_nodes, nodes_dist

In [28]:
def nodes_shortest_path(previous_nodes, start_node, target_node):
    path = []
    node = target_node
    
    while node != start_node:
        if node == None:
            return []
        path.append(node)
        node = previous_nodes[node]
        
    # Add the start node manually
    path.append(start_node)
    
    return list(reversed(path))

In [29]:
ehsi , calcl = shortest_path_Dijkstra(d_merge, '18313')

In [30]:
# peso totale con cui vado da '18313' a "19964"
min_dis_dijk = calcl["19964"]

In [31]:
# path completo dall'inizio alla fine
nodes_shortest_path(ehsi,'18313', "19964")

['18313', '28258', '6782', '19964']

In [32]:
def norm_closeness_centrality(dict_graph, v):
    
    # Create a list with all the nodes except the one considered
    nodes = list(dict_graph.keys())
    nodes.remove(v)
    
    # Iterate and calculate the denominator adding all the distances calculated with Dijkstra
    denom = 0
    _, dists = shortest_path_Dijkstra(dict_graph, v)
    
    for u in nodes:
        denom += dists[u]
        
    # Return the result
    return (n_users(dict_graph)) / denom

In [33]:
norm_closeness_centrality(d_merge, "18313")

0.0

In [34]:
norm_closeness_centrality(d_merge, "19964")

0.0

#### Page rank

The page rank algorithm will be implemented with Random Surfer Model in which we will use a value $\alpha$ to adjust all the points in the matrix of edges(a matrix where there are ones if the edge exists and zeros otherwise).
The matrix of edges could be full of zeros, so in order to save space in memory, we will not store it. We simply modify our dictionary graph, changing the weights and replacing each with:

\begin{equation}
\frac{\alpha}{|edges(v)|}+ (1-\alpha)\cdot w_{i}
\end{equation}

where $n$ is the node key in the dictionary, so $v \in V$ and $w_{i}$ is the weight of the $i-th$ edge.

For each other node where there is a zero in our "ideal" matrix, we will replace it simply with:
\begin{equation}
\frac{\alpha}{|edges(v)|}
\end{equation}

But of course it's not necessary to add it to the dictionary, since we write only the existing edges. We will assume that if an edge doesn't exist, the probability to go random to another not necessary connected node is $\frac{\alpha}{|edges(v)|}$

In [35]:
def get_parents(dict_graph, node):
    out = []
    for u in list(dict_graph.keys()):
            if u != node and (node in list(dict_graph[u].keys())):
                out.append(u)
    return out

In [36]:
def get_children(dict_graph, node):
    return list(dict_graph[node].values())

In [108]:
pagerank_dict = dict.fromkeys(d_merge.keys(), 0)

In [64]:
def update_pagerank(dict_graph, node, alpha, pagerank):
    parents = get_parents(dict_graph, node)
    pagerank_sum = sum((pagerank[node] / len(get_children(dict_graph, node))) for node in parents)
    random_walk = alpha / n_users(dict_graph)
    pagerank[node] = random_walk + (1 - alpha) * pagerank_sum

In [79]:
def PageRank_one_iter(dict_graph, alpha, pagerank):
    nodes = list(dict_graph.keys())
    for node in tqdm(nodes):
        update_pagerank(dict_graph, node, alpha, pagerank)

In [98]:
# alpha = 0,85 is optimal value for the damping factor
def page_rank(dict_graph, pagerank, alpha=0.85):
    PageRank_one_iter(dict_graph, alpha, pagerank)

In [109]:
for i in range(10):
    if i == 1:
        page_rank(d_merge, pagerank_dict)
        prev = list(pagerank_dict.values())
    if i > 1:
        page_rank(d_merge, pagerank_dict)
        cont = 0
        for pr, actual in zip(prev, list(pagerank_dict.values())):
            if pr == actual:
                cont += 1
        print("Convergence for --> " + str(cont) + " values.")
        prev = list(pagerank_dict.values())

100%|██████████| 5134/5134 [00:15<00:00, 324.24it/s]


Convergence for --> 1526 values.


100%|██████████| 5134/5134 [00:15<00:00, 321.74it/s]


Convergence for --> 1600 values.


100%|██████████| 5134/5134 [00:16<00:00, 312.85it/s]


...

Convergence for --> 1606 values.


100%|██████████| 5134/5134 [00:16<00:00, 305.13it/s]


Convergence for --> 1617 values.


100%|██████████| 5134/5134 [00:16<00:00, 320.39it/s]

Convergence for --> 1639 values.


#### Betweeness

As defined in *Introduction to Graph Concepts for Data Science of Aris Anagnostopoulos*, the Betweeness centrality of node $v$ is:

\begin{equation}
\sum_{u, w \epsilon V \backslash \{v\}} \frac{g_{u w}^v}{g_{uw}} \frac{2}{|V|^2 - 3|V| + 2} 
\end{equation}


with $g_{uw}$ the shortest path that connects nodes $u$ and $w$ and $g_{u w}^v$ the set of those shortest paths between u and w that contain node $v$.

In [111]:
# This function is a recursive implementation of BFS that finds all the paths between a start and an end node, but it will return only the ones that are minimum paths.
 
def find_all_shortest_paths(dict_graph, start_node, end_node, dijkstra_dist, path = []):
    
    # Initialize the list to do initial checks
    path = []
    
    # Initialize the total weight
    tot_weight = 0
    
    # Add to the path the starting node (after that in the recursion this list is used to track all the visited nodes)
    path = path + [start_node]
    
    # If the start is equal to the end
    if start_node == end_node:
        return [path]
    
    # If the node is not in the graph, we return an empty list
    if start_node not in dict_graph:
        return []
    
    # Create the final list that contains all the paths
    shortest_paths = []
    
    # Iterate over the neighbours of the start-node  
    for node in dict_graph[start_node]:
        # If the node is not in the path
        if node not in path:
            # Recursive call the function to apply BFS
            newpaths = find_all_shortest_paths(dict_graph, node, end_node, path)
            # Iterating over the lists of paths founded
            for newpath in newpaths:
                # Check for each node of this path if it is a minimum path, based on the distance calculated by dijkstra
                for i in range(0, len(newpath)-1): 
                    tot_weight += d[newpath[i]][newpath[i+1]]["weight"]
                # If this path is a mimum path, then append to the final list of lists
                if tot_weight == dijkstra_dist:
                    shortest_paths.append(newpath)
                    # Re-initialize the total weight
                    tot_weight = 0
    return shortest_paths

In [263]:
def betweeness(dict_graph, v):
    
    # Create a list with all the nodes except the one considered
    nodes = list(dict_graph.keys())
    nodes.remove(v)
    
    const = (2 / ((n_users(dict_graph))**2 - 3*(n_users(dict_graph)) + 2))
    result = 0
    
    # Iterating the list with a double for-loop
    for u, w in tqdm(zip(nodes, nodes)):
            g_u_v_w = 0
            g_u_w = 0
            previous_nodes, nodes_dist = shortest_path_Dijkstra(dict_graph, u)
            path = nodes_shortest_path(previous_nodes, u, w)
            # If the shortest path between u and w exists
            if nodes_dist[w] != float('inf') and nodes_dist[w] != 0 :
                g_u_w = nodes_dist[w]
                # And if exists and node v is in the path...
                if v in path:
                    g_u_v_w += 1
                result += (g_u_v_w / g_u_w) * const
    return result

In [ ]:
lista = find_all_shortest_paths(d_merge, "18313", "19964", min_dis_dijk)

In [115]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(5200)
print(sys.getrecursionlimit())

5200
5200


In [254]:
prova = nx.betweenness_centrality(merged)
prova["18313"]

0.0008620370926482761

In [ ]:
def F2_BestUser(Node, TimeInterval, Metric, Graph = MergedGraphs):
    """
    Input:
    A user/node
    A tuple of two dates in ISO format representing an interval of time, e.g. (2015-12-04, 2016-12-04) 
    An integer corresponding to the following metrics: 
      1 -> Betweeness 
      2 -> PageRank
      3 -> ClosenessCentrality 
      4 -> DegreeCentrality
    The graph on which to perform the analysis.

    Output:
    The value of the given metric applied over the complete graph for the given interval of time.
    """

    TimeInterval = tuple(map(dt.fromisoformat, TimeInterval)) # converting time interval into datetime format
    IntToMetric = {1: Betweeness, 2: PageRank, 3: ClosenessCentrality, 4: DegreeCentrality} # dictionary associating the
                                                                                           # input integer to the
                                                                                           # corresponding metric function

    TimeIntervalGraph = Graph.copy() # creating a copy on which to perform the time filtering
    TimeIntervalGraph.remove_edges_from([(n1, n2) for n1, n2, time in TimeIntervalGraph.edges(data="time") 
                                        if (time >= TimeInterval[0] & time <= TimeInterval[1])]) # removing the edges that not belong
                                                                                                 # to the input time interval  

    return IntToMetric[Metric](TimeIntervalGraph, Node)